## SQE with shells
Like for EEM, we implement SQE with polarizable shell particles. Since shells have fixed charges in Alexandria, the charge transfer elements $p_{ij}$ work between core charges $q^c$ only. To start with, we still have the core charge:

$q_i^c = \displaystyle{\frac{q_{tot}}{N}} + \sum\limits_{i,j}^{bonds} p_{ij}$

The energy now includes the shell charges $q_i^s$ explicitly:

$E_{SQE} = \sum\limits_{i=1}^{N} \left[\left(q_i^s+\frac{q_{tot}}{N} + \sum\limits_{k=1}^{M_i} p_{ik}\right)\left(\chi_i + \frac{1}{2} \eta_i \left[q_i^s+\frac{q_{tot}}{N} + \sum\limits_{k=1}^{M_i} p_{ik}\right]\right) + \frac{1}{2}\left(\frac{q_{tot}}{N} + \sum\limits_{k=1}^{M_i} p_{ik}\right)\left[\sum\limits_{l=1}^{N}\left[\frac{q_{tot}}{N} + \sum\limits_{m=1}^{M_l} p_{lm}\right]J_{il}^{cc}+ \sum\limits_{l\ne i}^{N}q_l^s J_{il}^{cs}\right] + \frac{1}{2}q_i^s\sum\limits_{l\ne i}^{N}q_l^s J_{il}^{ss}\right]+\frac{1}{2}\sum\limits_{i,j}^{M} \left[\zeta_{ij}p_{ij}^2+\Delta\chi_{ij}\left(\left[q_i^s + \frac{q_{tot}}{N} + \sum\limits_{k=1}^{M_i} p_{ik}\right]-\left[q_j^s + \frac{q_{tot}}{N} + \sum\limits_{m=1}^{M_j} p_{jm}\right]\right)\right] $

and our task now is to find the $p_{ij}$ that minimize $E_{SQE}$. Please note that the all summations are over atoms $i,j,k,l$. Finally, we again take the derivative and equate it to zero:

$0 = \displaystyle{\frac{\partial E_{SQE}}{\partial p_{ij}}} = \left(\chi_i - \chi_j+\frac{q_{tot}}{N}\left(\eta_i-\eta_j\right)+\sum\limits_{k=1}^{M_i}\Delta\chi_{ik}-\sum\limits_{k=1}^{M_j}\Delta\chi_{jk}+\frac{1}{2}\sum\limits_{l\ne i}^{N}q_l^s J_{il}^{cs}-\frac{1}{2}\sum\limits_{l\ne j}^{N}q_l^s J_{jl}^{cs} +  \frac{q_{tot}}{N}\left(\sum\limits_{l=1}^{N}J_{il}^{cc}-  \sum\limits_{l=1}^{N}J_{il}^{cc}\right)   \right) + \frac{1}{2}\left(\sum\limits_{l=1}^{N}J_{il}^{cc}\sum\limits_{m=1}^{M_l} p_{lm} - \sum\limits_{i=1}^{N}J_{li}^{cc}\sum\limits_{k=1}^{M_i} p_{ik}\right) + p_{ij}\zeta_{ij}$


Here too, we have the same matrix as in the non-polarizable case, but the right-hand side (RHS) of the equation has been extended with two pairs of terms due the shells. The resulting RHS is similar to the EEM case. **There may still be things missing here**. This is a guess:


$R_{ij} = R_j - R_i$
with

$R_j = \chi_j + \sum\limits_{k=1}^{M_j}\Delta\chi_{jk} +\frac{q_{tot}}{N}\sum\limits_{l=1}^N J_{jl}^{cc} + \frac{1}{2}\sum\limits_{l \ne j}^Nq_l^s J_{jl}^{cs} + q_j^s\eta_j .$

and likewise for $R_i$. In the two summations we assume that $J_{xx} = \eta_x$.

In [6]:
import numpy as np
from scipy.linalg import lstsq
import math
from charge_utils import *
from test_systems import *

def calcRHS(names, bonds, JEEM, qtotal,  verbose, model, shells, coords, qshell, shellx):
    Natoms = len(names)
    Nbond = len(bonds)
    ##### calc corrected chi ####
    chi_corr = []
    chi = get_chi(model)
    eta = get_eta(model)
    for i in range(Natoms):
        chi_corr.append(chi[names[i]])
        for k in range(Nbond):
            ai  = names[bonds[k][0]]
            aj  = names[bonds[k][1]]
            if bonds[k][0] == i:
                chi_corr[i] += getDeltaChi(ai, aj, model)
            if bonds[k][1] == i:
                chi_corr[i] -= getDeltaChi(ai, aj, model)
        for l in range(Natoms):
            chi_corr[i] += 0.5*JEEM[i][l]*(qtotal/Natoms)
        if shells:
            Jcs          = calcJcs(i, names, coords, shellx, qshell, model)
            if verbose:
                print("Jcs: %g" % (Jcs))
            chi_corr[i] += eta[names[i]]*qshell[i] + 0.5*Jcs

    #### calc EN diff for the bonds ####
    RHS = np.zeros(Nbond, dtype=float)
    for i in range(Nbond):
        RHS[i] = chi_corr[bonds[i][1]] - chi_corr[bonds[i][0]]
    if verbose or True:
        print("chi_corr: {}".format(chi_corr))
        print("RHS: {}".format(RHS))
    return RHS

def solveSQE(names, coords, bonds, verbose, qtotal, model, shells, qshell):
    JEEM, JSQE = calcJSQE(names, coords, bonds, verbose, model)
    shellmin  = ShellMinimize(names, coords, qshell, model)
    N  = names.shape[0]
    q0 = np.zeros(N)
    for i in range(N):
        q0[i] = qtotal/N-qshell[i]
    print("q0: {}".format(q0))
    chi2old   = shellmin.minimize(q0)
    iter      = 0
    toler     = 0.000001
    converged = False
    while not converged and iter < 50:
        rhs  = calcRHS(names, bonds, JEEM, qtotal, verbose, model, shells, coords, qshell, shellmin.shellX)
        pij  = np.linalg.solve(JSQE, rhs)
        if verbose:
            print("pij: {}".format(pij))
        q =  np.zeros(coords.shape[0], dtype=float)
        for n in range(len(bonds)):
            bi = bonds[n][0]
            bj = bonds[n][1]
            q[bi] += pij[n]
            q[bj] -= pij[n]
        Natoms = names.shape[0]
        for i in range(Natoms):
            q[i] += qtotal/Natoms-qshell[i]
        if shells:
            # Then minimize shells
            chi2 = shellmin.minimize(q)
            rmsd = shellmin.rmsd
            print("rmsd %10g energy %10g" % (rmsd, chi2))
            if abs(chi2-chi2old) < toler:
                converged = True
            else:
                chi2old = chi2
        else:
            converged = True
        if verbose or debug:
            print("JSQE = \n{}".format(JSQE))
            print("rhs = {}".format(rhs))
            print("q = {}".format(q))
        iter += 1


    print("qSQE: {}".format(q))
    return q, shellmin.shellX

def run_compound(molname, verbose, shells):
    mol = get_system(molname)
    print("\n%s" % molname)
    for bonds in mol["bonds"]:
        q, shellX = solveSQE(mol["names"], mol["coords"], bonds, verbose, mol["qtotal"], "ACS-pg", shells, mol["qshell"])
    printDipole(q, mol["coords"], mol["atomnr"], shells, mol["qshell"], shellX)

verbose = False
shells = True

for compound in [ "methanol", "water", "acetate", "carbon-monoxide" ]:
    run_compound(compound, verbose, shells)


methanol
q0: [2. 1. 2. 1. 1. 1.]
chi_corr: [-39.3414185113906, -34.07186950039013, -38.42144403933564, -35.08620059173808, -35.08621985542398, -34.856745165629036]
RHS: [ 0.91997447 -5.26954901  3.33522418 -3.33524345 -3.56469887]
rmsd  0.0140424 energy   -181.025
chi_corr: [-39.425812982764356, -34.11630045377535, -38.45936155859383, -35.08410097635405, -35.084156855037406, -34.889627535896224]
RHS: [ 0.96645142 -5.30951253  3.3752047  -3.37526058 -3.56973402]
rmsd 0.000305245 energy   -180.991
chi_corr: [-39.42747023908592, -34.117493446621744, -38.459198296321645, -35.083167257209354, -35.08322375191349, -34.889563407836384]
RHS: [ 0.96827194 -5.30997679  3.37597454 -3.37603104 -3.56963489]
rmsd 1.05317e-05 energy    -180.99
chi_corr: [-39.42752324709859, -34.11754178042592, -38.45918205157382, -35.083131114574854, -35.083187622598025, -34.88954302630222]
RHS: [ 0.9683412  -5.30998147  3.37599443 -3.37605094 -3.56963903]
rmsd 3.95005e-07 energy    -180.99
chi_corr: [-39.42752524796